In [1]:
import ruptures as rpt  # our package
from ruptures.metrics import hausdorff
import pandas as pd
import concurrent.futures
infile="/Users/irffanalahi/Research/Research_code/gitignorefolder/DMRrelated/ITGAE_patternrecognition/changepointtest/preproocess_Develop/ITGAEandERICH1_cin_nr0.5_imputed_rowmean.txt_compdeltafor_CD8TIL.txt"
minCpG=3
outfile=infile+"_mincpg_"+str(minCpG)+"pelt.txt"
penalty_value = 1 
indf=pd.read_csv(infile,sep="\t",index_col=0)

indf.head()

,CD8TIL-CD4bluourPBL,CD8TIL-CD8bluourPBL,CD8TIL-nB,CD8TIL-NK,CD8TIL-MnbluourPBL,CD8TIL-iDC,CD8TIL-mDC,CD8TIL-mNeu,CD8TIL-cB,CD8TIL-MEL_TUMOR
position,,,,,,,,,,
chr17:3800170,-0.460267,-0.369467,-0.520667,-0.571000,-0.600067,-0.563000,-0.604667,-0.453333,-0.165167,0.000667
chr17:3800341,-0.408067,-0.392067,-0.378000,-0.405333,-0.394267,-0.404333,-0.385333,-0.388667,-0.405000,-0.152000
chr17:3800417,-0.380200,-0.279200,-0.376000,-0.319667,-0.355400,-0.339000,-0.291667,-0.318333,-0.293333,-0.184667
chr17:3800454,-0.369400,-0.298200,-0.406333,-0.355333,-0.360000,-0.359667,-0.289333,-0.321000,-0.277667,-0.258667
chr17:3800480,-0.245800,-0.246800,-0.250000,-0.251333,-0.262200,-0.165000,-0.218333,-0.233000,-0.192000,0.013667


In [2]:
def segment_generator():
    allindex=indf.index.tolist()
    allindexchrom=list(set([i.split(':', 1)[0] for i in allindex]))
    segmentlist=[]
    for indexchrom in allindexchrom:
        segmentlist.append(indf[indf.index.str.contains(indexchrom)])
    
    return segmentlist
def pelt_result(signal_pelt):
    algo_python = rpt.Pelt(model="rbf", jump=1, min_size=minCpG).fit(signal_pelt)
    bkps_python = algo_python.predict(pen=penalty_value)
    forindexgenerate=bkps_python
    if forindexgenerate[-1]==len(signal_pelt.index):
        forindexgenerate[-1]=forindexgenerate[-1]-1
    return (signal_pelt.index[forindexgenerate]).tolist()

In [3]:
signallist=segment_generator()
multiresult = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    processlist=[]
    for signal in signallist:
        processlist.append(executor.submit(pelt_result,signal))
        #algo_python = rpt.Pelt(model="rbf", jump=1, min_size=minCpG).fit(signal)  # written in pure python
    for process in concurrent.futures.as_completed(processlist):
        multiresult.append(process.result())

In [4]:
flat = [x for sublist in multiresult for x in sublist]

with open(outfile, 'w') as f:
    for item in flat:
        f.write("%s\n" % item)
